In [ ]:
def get_avg_log_p_of_option(inputs_pretokenized, word_and_punc_pair):
    input_ids = tokenizer(inputs_pretokenized, return_tensors="pt").input_ids.to("cuda")
    # print(data_appended[0]['inputs_pretokenized'])
    # labels = tokenizer("<extra_id_0> " + word_and_punc_pair + " <extra_id_1>", return_tensors="pt").input_ids.to("cuda")
    labels = tokenizer("<extra_id_0> " + word_and_punc_pair, return_tensors="pt").input_ids.to("cuda")
    # HIGHLIGHT
    labels = labels[:, :-1]
    # print("<extra_id_0> " + id_to_word_and_punc_pairs_processed[0][2] + " <extra_id_1>")
    outputs = model(input_ids, labels=labels)
    # logits = outputs.logits
    # loss = loss_fn(logits.view(-1, logits.shape[-1])[1:labels.shape[1]-2], labels.view(-1)[1:labels.shape[1]-2])
    return -outputs.loss

In [ ]:
# count the number of correct predictions again using get_avg_log_p_of_options and ids_to_word_and_punc_pairs_processed
count_correct_avg_log_p_reranking = 0
for example_index in tqdm(range(1)): # len(data_appended)
    input_string = data_appended[example_index]['inputs_pretokenized']
    word_and_punc_pair_avg_log_p_max = -10000000
    best_word_and_punc_pair =  ""
    for word_and_punc_pair in id_to_word_and_punc_pairs_processed[example_index]:
        avg_log_p = get_avg_log_p_of_options(input_string, word_and_punc_pair)
        # print(avg_log_p)
        if avg_log_p > word_and_punc_pair_avg_log_p_max:
            word_and_punc_pair_avg_log_p_max = avg_log_p
            best_word_and_punc_pair = word_and_punc_pair
    best_word = best_word_and_punc_pair[:-1]
    print(best_word)
    print(best_word_and_punc_pair)
    if best_word == data_appended[example_index]['targets_pretokenized'][0]:
        count_correct_avg_log_p_reranking += 1

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
import os
# os.environ["TRANSFORMERS_CACHE"] = "/work/09127/tomyoung/ls6/LLM_cache/models--google--ul2"

# cache_dir='./models--google--ul2',
model = T5ForConditionalGeneration.from_pretrained("google/ul2", cache_dir='/work/09127/tomyoung/ls6/LLM_cache/google-ul2/', low_cpu_mem_usage=True, torch_dtype=torch.bfloat16).to("cuda")
model.parallelize()                                                                                                  
tokenizer = AutoTokenizer.from_pretrained("google/ul2")

input_string = "[NLU] Mr. Dursley was the director of a firm called <extra_id_0>, \
    which made <extra_id_1>. He was a big, solid man with a bald head. \
        Mrs. Dursley was thin and <extra_id_2> of neck, which came in very useful as she spent \
            so much of her time <extra_id_3>. The Dursleys had a small son \
                called Dudley and <extra_id_4>"                                           

print(sum(p.numel() for p in model.parameters()))
# 19,459,613,696
# inputs = tokenizer(input_string, return_tensors="pt", add_special_tokens=False).input_ids.to("cuda")

# outputs = model.generate(inputs, max_length=200)

# print(tokenizer.decode(outputs[0]))
# -> "<pad><extra_id_0> Burrows<extra_id_1> a lot of money from the manufacture of a product called '' Burrows'''s ''<extra_id_2> had a lot<extra_id_3> looking down people's throats<extra_id_4> a daughter called Petunia. Dudley was a very stupid boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat, ugly boy who was always getting into trouble. He was a big, fat,"


In [ ]:
# /work/09127/tomyoung/ls6/model_inputs_1_31am.pt
model_inputs_1_31_am = torch.load('/work/09127/tomyoung/ls6/model_inputs_1_31am.pt')
encoder_outputs_5_13am = torch.load('/work/09127/tomyoung/ls6/encoder_outputs_5_13am.pt')
# torch.save(model.state_dict(), '/work/09127/tomyoung/ls6/model_state_dict_1_31am.pt')

In [ ]:
# model_inputs_1_31_am: encoder_outputs, from generate, from model_kwargs, by passing prompt to the encoder
outputs = model(encoder_outputs=model_inputs_1_31_am['encoder_outputs'], decoder_input_ids=model_inputs_1_31_am['decoder_input_ids'])
# model_inputs_1_31_am
outputs

In [ ]:
'''Toy example'''
input_string = "[NLU] Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, solid man with a bald head. Mrs. Dursley was thin and blonde and more than the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbours. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere <extra_id_0>"                                               
inputs = tokenizer(input_string, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, max_length=200, num_beams=10, num_return_sequences=10)
for i in range(1):
    print(tokenizer.decode(outputs[i]))

In [ ]:
# generate for 0th example
input_string = data_appended[0]['inputs_pretokenized']
inputs = tokenizer(input_string, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, max_length=100, num_beams=100, num_return_sequences=100)
options = [tokenizer.decode(outputs[i]) for i in range(100)]
for i in range(100):
    print(tokenizer.decode(outputs[i]))

In [ ]:
# k = 0
print(len(outputs['scores']))
for k in range(len(outputs['scores'])):
    outputs['scores'][k]
    probs = torch.softmax(outputs['scores'][k], dim=-1)
    best_id = torch.argmax(probs)
    print(best_id)
    print(tokenizer.decode(best_id))
    print(probs[0][best_id])

# probs[0][3074]
# [1]
# torch.exp(outputs['scores'][1])
# torch.argmax(torch.exp(outputs['scores'][0]))
# # top 5
# torch.topk(torch.exp(outputs['scores'][0]), 5)

# torch.sum(torch.exp(outputs['scores'][0]))

In [ ]:
import math
# training
input_ids = tokenizer("[NLG] A man is having a bun for <extra_id_0>", return_tensors="pt").input_ids.to("cuda")
labels = tokenizer("<extra_id_0> lunch. A man is having a bun for lunch. A man is having a", return_tensors="pt").input_ids.to("cuda")
outputs = model(input_ids=input_ids, labels=labels)
loss = outputs.loss
logits = outputs.logits

# recover likelihood from loss
likelihood = math.exp(-loss)
likelihood